## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

## Faz download e carrega o dataset

In [367]:
%pip install --upgrade torchtext
%pip install --upgrade torch
%pip install --upgrade scikit-learn
%pip install --upgrade numpy
%pip install --upgrade lightning

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [368]:
import os

if not os.path.isfile("67724.txt.utf-8"):
    !curl -LO https://www.gutenberg.org/ebooks/67724.txt.utf-8

if not os.path.isfile("67725.txt.utf-8"):
    !curl -LO https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [369]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")
len(paragraphs)

4969

In [370]:
import re

def clean_text(text):
  # Aplicando lower case em todo o texto
  text = text.lower()
  # Removendo quebras de linha
  text = text.replace("\n", " ")
  text = text.replace("\n\n", " ")
  # Removendo caracteres especiais @, #, $, %, _, =, (, )
  text = re.sub(r'[@#$%_=()]', '', text)
  # Substituindo caracteres numéricos por [NUM]
  text = re.sub(r'\d+', '[num]', text)
  # Removendo espaços múltiplos
  text = re.sub(r'\s+', ' ', text)
  return text

cleaned_paragraphs = [clean_text(paragraph) for paragraph in paragraphs if paragraph.strip()]

print(f"""Sample do texto após ser processado: \n
---
{cleaned_paragraphs[150]}
---
""")

print(f"Tamanho da lista de parágrafos processados: {len(cleaned_paragraphs)}")

Sample do texto após ser processado: 

---
--para vós, não duvido; mas isto não é razão de que o seja para outros.
---

Tamanho da lista de parágrafos processados: 4892


## Análise do dataset

In [371]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(r'\w+|\S', text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12430

## Criando um vocabulário

In [372]:
vocab_size = 10000
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}
inverted_vocab = {index: word for word, index in vocab.items()}

print("Most frequent words:", most_frequent_words[:10])
print("Least frequent words:", most_frequent_words[-10:])

print(len(vocab))
print(vocab)

Most frequent words: ['.', ',', '-', 'a', 'que', 'o', 'de', 'e', 'se', ';']
Least frequent words: ['trocadas', 'luzir', 'esgueirando', 'chegára', 'dezenoze', 'sobrão', 'concebi', 'servi', 'lealmente', 'cova']
10000
{'.': 1, ',': 2, '-': 3, 'a': 4, 'que': 5, 'o': 6, 'de': 7, 'e': 8, 'se': 9, ';': 10, 'um': 11, 'do': 12, 'não': 13, 'uma': 14, 'da': 15, 'os': 16, 'com': 17, 'sua': 18, 'para': 19, 'seu': 20, '!': 21, 'pery': 22, 'as': 23, 'em': 24, 'no': 25, '?': 26, 'por': 27, 'ao': 28, 'como': 29, 'lhe': 30, 'd': 31, 'á': 32, 'tinha': 33, 'era': 34, ':': 35, 'cecilia': 36, 'na': 37, 'é': 38, 'sobre': 39, 'mas': 40, 'elle': 41, 'the': 42, '[': 43, ']': 44, 'num': 45, 'dos': 46, 'indio': 47, 'me': 48, 'seus': 49, 'mais': 50, 'antonio': 51, 'quando': 52, 'alvaro': 53, 'disse': 54, 'das': 55, 'vos': 56, 'of': 57, 'ella': 58, 'senhora': 59, 'olhos': 60, 'te': 61, 'menina': 62, 'pela': 63, 'tu': 64, "'": 65, 'depois': 66, 'nos': 67, 'isabel': 68, 'havia': 69, 'gutenberg': 70, 'fidalgo': 71, 'c

In [392]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r'\w+|\S', sentence.lower())]

# Definindo a função de decodificação
def decode_sentence(encoded_sentence):
    sentence = []
    for encoded_token in encoded_sentence:
        for word, token in vocab.items():
            if encoded_token == token:
                sentence.append(word)

    return sentence

In [393]:
encoded_paragraph = encode_sentence(cleaned_paragraphs[20], vocab)
decoded_paragraph = decode_sentence(encoded_paragraph)

In [375]:
print(f"""Pragraph: {cleaned_paragraphs[20]}
Encoded: {encoded_paragraph}
Decoded: {decoded_paragraph}
""")

Pragraph:  publicando este livro em [num], se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
Encoded: [6569, 150, 4388, 24, 43, 45, 44, 2, 9, 54, 121, 281, 271, 2672, 14, 1078, 6570, 2, 5, 198, 142, 292, 6, 2269, 9, 6571, 4, 2675, 1]
Decoded: publicando este livro em [ num ] , se disse ser aquella primeira edição uma prova typographica , que algum dia talvez o autor se dispuzesse a rever .



## Classe do dataset

In [376]:
context_size = 5 # 5 palavras de entrada. O target é a próxima palavra

import torch
from torch.utils.data import Dataset, DataLoader

class GuaraniDataset(Dataset):
    def __init__(self, text, context_size, vocab):
        self.text = text
        self.context_size = context_size
        self.vocab = vocab
        self.input_target_pairs = self.prepare_dataset()

    def prepare_dataset(self):
        """
        Cria pares de entrada e alvo com base no tamanho da janela de contexto.
        """
        input_target_pairs = []
        for paragraph in self.text:
            tokens = encode_sentence(paragraph, self.vocab)
            if len(tokens) >= self.context_size:
              for i in range(len(tokens) - self.context_size):
                  context = tokens[i:i+self.context_size]
                  target  = tokens[i+self.context_size]
                  input_target_pairs.append((torch.tensor(context), torch.tensor(target)))
        return input_target_pairs

    def __len__(self):
        """
        Retorna o tamanho do dataset.
        """
        return len(self.input_target_pairs)

    def __getitem__(self, idx):
        """
        Retorna um par de entrada e alvo no índice especificado, convertidos em tensores.
        """
        return self.input_target_pairs[idx]

In [377]:
from sklearn.model_selection import train_test_split

context_size = 10
# Dividindo os pares de entrada-alvo em conjuntos de treino e validação
train_data, val_data = train_test_split(
    cleaned_paragraphs,
    test_size=0.2,
    random_state=18)

#Verificando o tamanho dos vetores
print(f"""
---
Conjunto Treinamento: {len(train_data)} [{round((len(train_data) / len(cleaned_paragraphs)) * 100)}%]
---
Conjunto Validação: {len(val_data)} [{round((len(val_data) / len(cleaned_paragraphs)) * 100)}%]
---
""")

train_data = GuaraniDataset(train_data, context_size, vocab)
val_data = GuaraniDataset(val_data, context_size, vocab)


---
Conjunto Treinamento: 3913 [80%]
---
Conjunto Validação: 979 [20%]
---



In [378]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
sample = next(iter(train_loader))

## Implementação da Autoatenção

In [379]:
import torch
import torch.nn.functional as F

embedding_dim = 64
embed = torch.nn.Embedding(num_embeddings=vocab_size + 1, embedding_dim=embedding_dim)

def C(word):
    return vocab.get(word, 0) 

# Implementação da auto-atenção usando loop
def self_attention_loop(seq):
    E = []
    for q in seq:
        scores = []
        for k in seq:
            score = torch.matmul(embed(torch.tensor(C(q))), embed(torch.tensor(C(k))).T)
            scores.append(score)
        probs = F.softmax(torch.stack(scores), dim=0)
        new_embedding = torch.zeros(embedding_dim)
        for v, p in zip(seq, probs):
            new_embedding += embed(torch.tensor(C(v))) * p
        E.append(new_embedding)
    return torch.stack(E)

# Implementação da auto-atenção usando operações matriciais
def self_attention_matrix(seq):
    def attention(Q, K, V):
        scores = torch.matmul(Q, K.transpose(-2, -1))
        probs = F.softmax(scores, dim=-1)
        E = torch.matmul(probs, V)
        return E

    indices = torch.tensor([C(w) for w in seq], dtype=torch.long)
    X = embed(indices)
    return attention(X, X, X)

# Testando as funções com uma sequência de palavras exemplo usando PyTorch
seq = ['w1', 'w2', 'w3']
embeddings_loop_torch = self_attention_loop(seq)
embeddings_matrix_torch = self_attention_matrix(seq)

assert torch.allclose(embeddings_loop_torch, embeddings_matrix_torch, atol=1e-6), "As implementações de loop e matricial são diferentes!"
print("Métodos equivalentes")


Métodos equivalentes


### Implementação das projeções lineares

In [380]:
import torch
import torch.nn.functional as F
D = 10

# Inicializando as matrizes de pesos das projeções linearmente
torch.manual_seed(0)
W_Q = torch.randn((D, D))
W_K = torch.randn((D, D))
W_V = torch.randn((D, D))
W_O = torch.randn((D, D))

# Implementação da auto-atenção usando loop com projeções lineares
def self_attention_loop_projection(seq):
  E = []
  for x in word_list:
    q = torch.matmul(x, W_Q)
    scores = []
    for xk in seq:
        k = torch.matmul(xk, W_K)

        score = torch.matmul(q, torch.transpose(torch.unsqueeze(k, 0), 0, 1))
        scores.append(score)
    probs = torch.softmax(torch.tensor(scores), dim = -1)

    e = 0
    for xv, p in zip(word_list, probs):
        v = torch.matmul(xv, W_V)
        e += v * p
    e = torch.matmul(e, W_O)
    E.append(e)
  return torch.stack(E)

# Implementação da auto-atenção usando operações matriciais com projeções lineares
def self_attention_matrix_projection(word_stack):

  q = torch.matmul(word_stack, W_Q)
  k = torch.matmul(word_stack, W_K)
  v = torch.matmul(word_stack, W_V)
  scores = torch.matmul(q, torch.transpose(k, -2, -1))
  probs = torch.softmax(scores, dim=-1)
  e_mtx = torch.matmul(probs, v)
  return torch.matmul(e_mtx, W_O)

l = 3
word_list = [torch.randn((D)) for i in range(l)]
seq = torch.stack(tuple(word_list))
embeddings_loop_proj = self_attention_loop_projection(seq)
embeddings_matrix_proj = self_attention_matrix_projection(seq)

# Verificação de igualdade das implementações
assert torch.allclose(embeddings_loop_proj, embeddings_matrix_proj, atol=1e-6), "As implementações de loop e matricial com projeções são diferentes!"
print("Métodos equivalentes")

Métodos equivalentes


## Model

In [381]:
import torch.nn as nn

import torch
import torch.nn as nn
import torch.nn.functional as F

# Implementação inspirada na do aluno Felipe Gabriel Brabes da Silva
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, embedding_dim, context_size):
        super(LanguageModel, self).__init__()

        self.embedding_dim = embedding_dim

        self.embedding_matrix = torch.nn.Parameter(torch.randn((vocab_size + 1, embedding_dim)))
        self.W_Q = torch.nn.Parameter(torch.randn((embedding_dim, embedding_dim)))
        self.W_K = torch.nn.Parameter(torch.randn((embedding_dim, embedding_dim)))
        self.W_V = torch.nn.Parameter(torch.randn((embedding_dim, embedding_dim)))
        self.W_O = torch.nn.Parameter(torch.randn((embedding_dim, embedding_dim)))

        torch.nn.init.kaiming_normal_(self.embedding_matrix)
        torch.nn.init.kaiming_normal_(self.W_Q)
        torch.nn.init.kaiming_normal_(self.W_K)
        torch.nn.init.kaiming_normal_(self.W_V)
        torch.nn.init.kaiming_normal_(self.W_O)

        self.positional_embedding = torch.nn.Parameter((70 ** ((torch.arange(0, embedding_dim//2).repeat_interleave(2))* -1/embedding_dim)) * torch.unsqueeze(torch.arange(0, context_size), 1).float(), requires_grad=False)
        self.positional_embedding[:, torch.arange(0, embedding_dim//2)*2] = torch.sin(self.positional_embedding[:, torch.arange(0, embedding_dim//2)*2])
        self.positional_embedding[:, 1 + torch.arange(0, embedding_dim//2)*2] = torch.cos(self.positional_embedding[:, 1 + torch.arange(0, embedding_dim//2)*2])

        self.l1 = nn.Linear(context_size * embedding_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, (vocab_size + 1))

        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.layer_norm = nn.LayerNorm((hidden_dim))

    def self_attention_matricial(self, word_stack, W_Q, W_K, W_V, W_O, embedding_size):

      Q = torch.matmul(word_stack, W_Q)
      K = torch.matmul(word_stack, W_K)
      V = torch.matmul(word_stack, W_V)
      scores = torch.matmul(Q, torch.transpose(K, -2, -1)) / embedding_size
      probs = torch.softmax(scores, dim=-1)
      E = torch.matmul(probs, V)
      return torch.matmul(E, W_O)


    def forward(self, words):
        x = self.embedding_matrix[words, :] + self.positional_embedding

        x = torch.flatten(self.self_attention_matricial(x, self.W_Q, self.W_K, self.W_V, self.W_O, self.embedding_dim), 1) #B, L*D

        x = self.tanh(self.l1(x))
        return self.l2(self.layer_norm(x))

In [382]:
embedding_dim = 64   # Dimensão do embedding
hidden_dim	  = 256*2

# Instância do modelo
model = LanguageModel(vocab_size, hidden_dim, embedding_dim, context_size)

print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

LanguageModel(
  (l1): Linear(in_features=640, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=10001, bias=True)
  (tanh): Tanh()
  (relu): ReLU()
  (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)
6116177


In [383]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]
output = model(input)

In [384]:
print(f"""
---
{output.argmax(dim=1)}
---
{target}
""")


---
tensor([3663, 3663,  881, 3663, 3663, 3663,  881, 4473, 3663, 4473, 4473, 3663,
        3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663,  881,
        3663, 4473, 3663,  881, 3663,  881, 3663, 4473])
---
tensor([   8,    7, 1381,    7,   61, 9535,    4,    1,   27,  196,    1,   12,
           7,   14,  162,   49,    1,    2,  299, 5157, 4861,    1,    2,  152,
          16,    1,   31,    6,   23,   49, 5953,   17])



In [385]:
output.argmax(dim=1)

tensor([3663, 3663,  881, 3663, 3663, 3663,  881, 4473, 3663, 4473, 4473, 3663,
        3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663, 3663,  881,
        3663, 4473, 3663,  881, 3663,  881, 3663, 4473])

In [386]:
target

tensor([   8,    7, 1381,    7,   61, 9535,    4,    1,   27,  196,    1,   12,
           7,   14,  162,   49,    1,    2,  299, 5157, 4861,    1,    2,  152,
          16,    1,   31,    6,   23,   49, 5953,   17])

## Training e Avaliação

In [387]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [388]:
import torch.optim as optim
import time
import numpy as np

# Definições iniciais
epochs = 10
lr = 0.08 # Defina uma taxa de aprendizado inicial
criterion = nn.CrossEntropyLoss()  # Função de perda de entropia cruzada para classificação
optimizer = optim.SGD(model.parameters(), lr=lr)  # Usando AdamW como otimizador
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Ajusta a taxa de aprendizado

best_val_loss = float('inf')  # Para monitorar a melhor loss de validação

# Loop de treinamento
for epoch in range(epochs):
    start_time = time.time()  # Start time of the epoch
    model.train()  # Coloca o modelo em modo de treinamento
    running_loss = 0.0
    for inputs, labels in train_loader:  # Supondo que 'train_loader' é seu DataLoader de treino
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)  # Faz a predição
        loss = criterion(outputs, labels)  # Calcula a perda

        optimizer.zero_grad()  # Zera os gradientes acumulados
        loss.backward()  # Propagação reversa
        optimizer.step()  # Atualiza os pesos

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)  # Calcula a loss média por época
    perplexity = np.exp(epoch_loss)  # Calcula a perplexidade para a época

    # Avaliação no conjunto de validação
    model.eval()  # Coloca o modelo em modo de avaliação
    val_running_loss = 0.0
    with torch.no_grad():  # Desativa o cálculo de gradientes
        for inputs, labels in val_loader:  # Supondo que 'val_loader' é seu DataLoader de validação
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_perplexity = np.exp(val_epoch_loss)  # Calcula a perplexidade para o conjunto de validação

    # Atualiza o scheduler
    #scheduler.step()

    # Verifica se a loss de validação melhorou
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Salva o modelo
    
    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch
    print(f'Epoch [{epoch+1}/{epochs}] | '
          f'Loss: {epoch_loss:.4f} | '
          f'Perplexity: {perplexity:.2f} | '
          f'Elapsed Time: {epoch_duration:.2f} sec | '
          f'Validation Loss: {val_epoch_loss:.4f} | '
          f'Validation Perplexity: {val_perplexity:.2f}')

Epoch [1/10] | Loss: 6.7020 | Perplexity: 814.06 | Elapsed Time: 5.75 sec | Validation Loss: 6.1359 | Validation Perplexity: 462.16
Epoch [2/10] | Loss: 6.1561 | Perplexity: 471.58 | Elapsed Time: 5.67 sec | Validation Loss: 5.6736 | Validation Perplexity: 291.09
Epoch [3/10] | Loss: 5.6335 | Perplexity: 279.63 | Elapsed Time: 5.05 sec | Validation Loss: 5.3662 | Validation Perplexity: 214.05
Epoch [4/10] | Loss: 5.3366 | Perplexity: 207.81 | Elapsed Time: 5.35 sec | Validation Loss: 5.2146 | Validation Perplexity: 183.93
Epoch [5/10] | Loss: 5.1258 | Perplexity: 168.31 | Elapsed Time: 5.11 sec | Validation Loss: 5.1315 | Validation Perplexity: 169.26
Epoch [6/10] | Loss: 4.9481 | Perplexity: 140.90 | Elapsed Time: 4.84 sec | Validation Loss: 5.0750 | Validation Perplexity: 159.97
Epoch [7/10] | Loss: 4.7860 | Perplexity: 119.83 | Elapsed Time: 5.06 sec | Validation Loss: 5.0264 | Validation Perplexity: 152.39
Epoch [8/10] | Loss: 4.6355 | Perplexity: 103.08 | Elapsed Time: 5.01 sec | 

## Exemplo de uso

In [403]:
import torch.nn.functional as F

# Implementação baseada na do aluno Otavio Cury Pontes
text = "As noites de um verão qualquer são boas para um"
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

def generate_text(model, vocab, text, context, max_length):
    cleaned_paragraphs = text.replace("\n", " ").lower()
    encoded_sentence = encode_sentence(text, vocab)

    for i in range(max_length):
        last_context = torch.tensor(encoded_sentence[-context:]).unsqueeze(0)
        last_context = last_context.to(device)
        with torch.no_grad():  # No need to track gradients during generation
            outputs = model(last_context)
            prob = F.softmax(outputs, dim=1)

        predicted_index = torch.multinomial(prob, 1)

        encoded_sentence.append(predicted_index.item())

    return encoded_sentence


context = 10
encoded_sentence = generate_text(model, vocab, text, context, max_length=40)
sentence = decode_sentence(encoded_sentence)
print(' '.join(sentence))

as noites de um verão qualquer são boas para um selvagem de alguem , e murmurava - lo que lhe partira porque vós da vida , esperava entre o quarto que estabelecer sagrada : ou que que vos pode soffrer adormeceu que entrar fixidade homem da noite . mas não
